In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 14 16:06:58 2019

@author: satida
"""


import numpy as np
order=4
from .Utils import deleteDoller,FarsiComparison,SortFa,WordTranslate
import persian

class Converter:
    
    def ToNorm(self,ref,K=4):
        self.Nref=dict()
        for i,r in enumerate(ref):
            self.Nref[i * 4]= r
        
        return list(self.Nref.keys())
    
    def ToRef(self,norm):
        return self.Nref[norm]
    
    
class Node:
    def __init__(self,root):
        self.root=root
        self.child=list()
        self.content=list()
        self.child_size=0
        self.content_size=0
    
    def insertContent(self,item):
        
        if len(self.content) < order -1 :
            self.content.append(item)
            self.content=SortFa(self.content[:])
            self.content_size=len(self.content)
            self.child_size=len(self.child)
            istidx=self.content.index(item)
            if istidx == 0:
                if self.content_size == 1:
                    self.child.append(Node(self))
                    self.child.append(Node(self))
                else :
                    self.child=[Node(self)]+self.child
            elif istidx == self.content_size -1:
                self.child.append(Node(self))
                
            else :
                self.child=self.child[:istidx]+[Node(self)]+self.child[istidx:]
            
            return True
        else :
            return False
    
    def insertChild(self,child):
        if self.child_size < len(self.content) + 1:
            self.child.append(child)
            self.child_size=len(self.child)
            return True
        else :
            return False
        
    def __getitem__(self,name:str):
        if name == "child":
            return self.child[:]
        elif name == "content":
            return self.content[:]
        
    def setNode(self,node):
        self.root=node.root
        self.child=node.child[:]
        self.content=node.content[:]
        self.child_size=node.child_size
        self.content_size=node.content_size
        return True
        
class Btree:
    def __init__(self,String,conv:Converter):
        self.String=String
        self.root=Node(0)
        self.Translate=WordTranslate(String).Translate
        self.FindMaxOnBlock=WordTranslate(String).FindMaxOnBlock
        self.ToRef=conv.ToRef
        
    
    def Search(self,node:Node,item):
        if node.content_size != 0:
            for i,cnt in enumerate(node.content):
                for word in self.Translate(self.ToRef(cnt)).split(","):
                    if item in word[:len(item)] :
                        return node,(node.content)
                    
                    
                if FarsiComparison(item) > FarsiComparison(self.FindMaxOnBlock(self.ToRef(cnt))) and i == (len(node.content)-1):
                    return self.Search(node.child[i+1],item)
                
                elif FarsiComparison(item) < FarsiComparison(self.FindMaxOnBlock(self.ToRef(cnt))) :
                    return self.Search(node.child[i],item)
        else:
            return False
    
    def __Insert(self,node:Node,item):
        if self.__hasChild(node) :
            for i,cnt in enumerate(node.content):
                
                if FarsiComparison(item) < FarsiComparison(cnt) :
                    temp=self.__Insert(node.child[i],item)
                    return self.__merge(temp,node)
                    
                    
                elif FarsiComparison(item) > FarsiComparison(cnt) and i == (len(node.content)-1):
                    temp =self.__Insert(node.child[i+1],item)
                    return self.__merge(temp,node)
                    
                
        elif node.content_size < order -1 and not self.__hasChild(node) :
            return node.insertContent(item)
        
        elif node.content_size == order -1  :
            if node.root == 0:
                return node.setNode(self.__Split(node,item))
            else :
                return self.__Split(node,item)
    
    def __merge(self,temp,node):
        if type(temp)== bool:
            return temp
        if node.insertContent(temp.content[0]):
            insidx= node.content.index(temp.content[0])
            node.child[insidx].setNode(temp.child[0])
            node.child[insidx+1].setNode(temp.child[1])
            return True
        else :
            return self.__SplitNode(node,temp)
        
    
    def __hasChild(self,node:Node):
        for i in node.child:
            if i.content_size > 0:
                return True
        return False                        
                    
    def __SplitNode(self,node:Node,temp:Node):
        content=node.content+temp.content
        content=SortFa(content)
        mid_index=len(content)//2
        mid=content[mid_index]
        father=Node(node.root)
        father.insertContent(mid)
        temp_idx=content.index(temp.content[0])
        children=node.child
        children=children[:temp_idx]+temp.child+children[temp_idx+1:]
        child_l=Node(father)
        
        for i in content[:mid_index]:
            child_l.insertContent(i)
        child_l.child=children[:mid_index+1]
        
        child_r=Node(father)
        
        for i in content[mid_index+1:]:
            child_r.insertContent(i)
        child_r.child=children[mid_index+1:]
        
        father.child[0].setNode(child_l)
        father.child[1].setNode(child_r)
        node.setNode(father)
        return father
        
        
    def __Split(self,node:Node,item):
        temp=node.content[:]
        temp.append(item)
        temp=SortFa(temp)
        
        mid=temp[len(temp)//2]
        father=Node(node.root)
        father.insertContent(mid)
        
        child_r=Node(father)
        child_l=Node(father)
        
        for i in temp[:len(temp)//2]:
            child_l.insertContent(i)
            
        for i in temp[len(temp)//2 + 1:]:
            child_r.insertContent(i)
            
        for i in range(len(temp[:len(temp)//2])):
            child_l.child[i]=node.child[i]
            
        for i in range(len(temp[len(temp)//2 +1:])):
            child_r.child[i]=node.child[i]
            
        father.child[0]=child_l
        father.child[1]=child_r
        
        return father                    


    def GetTree(self,root):    
        return root.child,root.content
        
    
    def insert(self,item):
        return self.__Insert(self.root,item)
    
    def ReturnAllValues(self,root:Node,res_words:list,res_ref:list):
        for c in root.content:
            content=self.Translate(self.ToRef(c)).split(",")
            for i,cnt in enumerate(content):
                if "$" in cnt: 
                    res_words.append(deleteDoller(cnt))
                    res_ref.append(c+i)
                else:
                    res_words.append(cnt)
                    res_ref.append(c+i)
                    
        if self.__hasChild(root):
            for child in root.child:
                self.ReturnAllValues(child,res_words,res_ref)
        else:
            return True
    
    
        




# =============================================================================
# import pickle
# 
# with open("Full.pkl","rb") as f:
#     dic=pickle.load(f)
# #init Btree
# tree=Btree(dic["String"])
# 
# for i in dic["indexes"]:
#     tree.insert(i)
# 
# x=tree.Search(tree.root,"خانه")
# temp=list()
# f=tree.ReturnAllValues(x[0],temp)
# res=list()
# t=""
# for t in temp:
#     if t.endswith("خانه"):
#         res.append(t)
# =============================================================================
#Insert sample persian Numbers

# =============================================================================
# 
# #Get btree root Node
# root=tree.root
# 
# #Init a list for save result
# result=list()
# tree.ReturnAllValues(root,result)
# 
# #Find needed word in tree and return right child node
# #Find 80* for example
# node=tree.Search(root,persian.convert_en_numbers("80"))
# 
# #You can find all result with found node as attribute
# res2=list()
# tree.ReturnAllValues(node[0],res2)
# res2=SortFa(res2)
# 
# =============================================================================










